In [ ]:
print('Hello World!')

In [ ]:
#Encrypting Labels

In [ ]:

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
# Ignore all FutureWarning warnings that might flood the console log
warnings.simplefilter(action='ignore', category=DeprecationWarning)
warnings.simplefilter(action='ignore', category=RuntimeWarning)

import os

# from encryption import encode, decode

import time
from utils import create_patches_v3, create_binary_mask, create_patches_v2, create_patches_v1, create_patches_coordinates, create_patches_coordinates_v1

# Loading directory
wsi_dir = "/root/code2024/dataset/1WSI/data/"
patch_size = 224
cpu_core = 12

# Saving directory
save_dir = wsi_dir

downsize = patch_size
patch_extraction_size = patch_size
mask_overlap = 80.0
batch_size = 64
cpu_workers = cpu_core
use_prob_threshold = 0.8 # None  # whether to give final prediction {0,1} based on certain probability

# torch.manual_seed(250)
print('--------------Start--------------')
# read the files
wsi_files = os.listdir(wsi_dir)
# get all files except temp directory containing patches
wsi_files = [f for f in wsi_files if f.endswith("svs") or f.endswith("ndpi") or f.endswith("mrxs")]

print(f"Total files in {wsi_dir} directory are {len(wsi_files)}")

path = os.path.join(wsi_dir, "cnn_ensemble_updated")
if not os.path.exists(path):
    os.mkdir(path)

lst_patching_time = []

stat_NMI_graph = [] 
stat_NMI_base = []
coords = []
new_file_paths = []
# start patching process over a certain WSI
for i, f in enumerate(wsi_files):

    st_binarymask = time.time()
    new_file_path = os.path.join(path, f.split(".")[0])
    print(new_file_path)

    # just take the name not extension
    if not os.path.exists(new_file_path):
        os.mkdir(new_file_path)
        
    # find binary mask to locate tissue on WSI            
    w, h = create_binary_mask(wsi_dir, f, new_file_path, downsize=patch_size)

    # print(w/224, h/224)
    et_binarymask = time.time()

    binarymask_processtime = et_binarymask - st_binarymask
    print(f"Time spent on create_binary_mask is {binarymask_processtime} for {f}")
        
    st_patching = time.time()
    patch_folder = os.path.join(new_file_path, "patches")
    
    if not os.path.exists(patch_folder):
    # # else:
        os.mkdir(patch_folder) 
    if os.path.exists(patch_folder):
    #     os.rmdir(patch_folder)
    # elif not os.path.exists(patch_folder):
    # # else:
    #     os.mkdir(patch_folder) 
        # coordinates, id_to_coordinates = create_patches_coordinates(wsi_dir, f, new_file_path, patch_folder, workers=cpu_workers,
                    #    patch_size=patch_extraction_size, mask_overlap=mask_overlap) #单进程
        # coordinates, id_to_coordinates, patches = create_patches_v1(wsi_dir, f, new_file_path, patch_folder, workers=cpu_workers,
                    #    patch_size=patch_extraction_size, mask_overlap=mask_overlap) #单进程
        
        # create_patches_v3(wsi_dir, f, new_file_path, patch_folder, workers=cpu_workers,
                    #    patch_size=patch_extraction_size, mask_overlap=mask_overlap) #多进程
        
        # 1. create coordinates with multi-processing
        res = create_patches_coordinates_v1(wsi_dir, f, new_file_path, workers=cpu_workers, patch_size=patch_extraction_size, mask_overlap=mask_overlap) #多进程
        
        # patches = create_patches_v2(wsi_dir, f, new_file_path, patch_folder, workers=cpu_workers,
            # patch_size=patch_extraction_size, mask_overlap=mask_overlap) #多进程
        # print(len(coordinates))
        # print(len(patches))
        # print(id_to_coordinates)

        # print("Coordinates Done!", len(res), res[:3])
        # coordinates = list(res.values())
        coordinates = res.values()
        ids = res.keys()
        # print("Coordinates Done!", len(res), len(coordinates), coordinates[:3])
        coords.append(coordinates)
        new_file_paths.append(new_file_path)
        

In [ ]:
print(new_file_paths)
print(len(coords))
# import networkx as nx




# Graph-based Split Data Pertubation

In [ ]:
import os
import fnmatch
def count_files_with_numeric_pattern(start_path, file_pattern):
    count = 0
    for root, dirs, files in os.walk(start_path):
        for file in files:
            if fnmatch.fnmatch(file, file_pattern):
                count += 1
                # print(os.path.join(root, file))  # 打印匹配的文件路径
    return count

In [ ]:
import pandas as pd
from encryption_r import encode, decode
    # r coordinates, new_file_path in zip(coords, new_file_paths): 
  
for new_file_path in new_file_paths: 
    graph_split_file = f'{new_file_path}/graph_split1/'
    os.listdir(graph_split_file)
    strategies = ['largest_first', 'random_sequential', 'smallest_last', 'independent_set', 
            'connected_sequential', 'saturation_largest_first']
    for my_strategy in strategies: 
        # my_strategy = 'smallest_last'
        # d, dataset_number = graph_coloring_based_splitting_visual1(graph_split_file, coordinates, patch_size, strategy=my_strategy)
        
        # 设置你的目标文件夹名称和文件模式
        graph_split_file = f'{new_file_path}/graph_split1/'
        file_pattern = f"*[0-9]{my_strategy}.csv"

        # 计算文件数量
        dataset_number = count_files_with_numeric_pattern(graph_split_file, file_pattern)
        print(my_strategy, dataset_number)
        # print(f"在 '{target_folder_name}' 文件夹中找到了 {file_count} 个符合 '{file_pattern}' 模式的文件。")
        for j in range(dataset_number):
            # sub data set
            origfpath = f'{graph_split_file}/graph_patchesnames_{str(j)}{my_strategy}.csv'
            encodefpath = f'{graph_split_file}{my_strategy}_R_encode_coordinates_{j}.csv'
            decodefpath = f'{graph_split_file}{my_strategy}_R_decode_coordinates_{j}.csv'

            print(new_file_path, my_strategy, j)
            xencry_list, yencry_list, means, X_centered, Y_centered, x_list, y_list, Rxencry_list, Ryencry_list = encode(origfpath, encodefpath) # type: ignore     
            # print(xencry_list, yencry_list, means, X_centered, Y_centered, x_list, y_list, Rxencry_list, Ryencry_list)
            xdecry_list, ydecry_list = decode(encodefpath, means, X_centered, Y_centered, decodefpath)

            perturbe_df = pd.read_csv(encodefpath)
            orig_df = pd.read_csv(origfpath)
            estimate_df = pd.read_csv(decodefpath)

            PerturbedList = perturbe_df["EncryPatchName"].tolist()
            origList = orig_df["PatchName"].tolist()
            estList = estimate_df["DecryPatchName"].tolist()
            # print(estList)



# reconstruct graph with decrypted lables

In [ ]:
from parse_coordinates import find_decrpytcoordinates, find_encrpytcoordinates


import pandas as pd
from encryption_r import encode, decode
    # r coordinates, new_file_path in zip(coords, new_file_paths): 

de_coords = []    
for new_file_path in new_file_paths: 
    graph_split_file = f'{new_file_path}/graph_split1/'
    os.listdir(graph_split_file)
    # strategies = ['largest_first', 'random_sequential', 'smallest_last', 'independent_set', 
            # 'connected_sequential', 'saturation_largest_first']
    strategies = ['saturation_largest_first']        
    for my_strategy in strategies: 
        # my_strategy = 'smallest_last'
        # d, dataset_number = graph_coloring_based_splitting_visual1(graph_split_file, coordinates, patch_size, strategy=my_strategy)
        
        # 设置你的目标文件夹名称和文件模式
        graph_split_file = f'{new_file_path}/graph_split1/'
        file_pattern = f"*[0-9]{my_strategy}.csv"

        # 计算文件数量
        dataset_number = count_files_with_numeric_pattern(graph_split_file, file_pattern)
        print(my_strategy, dataset_number)
        # print(f"在 '{target_folder_name}' 文件夹中找到了 {file_count} 个符合 '{file_pattern}' 模式的文件。")
        coordinates = []
        for j in range(dataset_number):
            # sub data set
            # origfpath = f'{graph_split_file}/graph_patchesnames_{str(j)}{my_strategy}.csv'
            # encodefpath = f'{graph_split_file}{my_strategy}_R_encode_coordinates_{j}.csv'
            decodefpath = f'{graph_split_file}{my_strategy}_R_decode_coordinates_{j}.csv'

            print(new_file_path, my_strategy, j)
            # xencry_list, yencry_list, means, X_centered, Y_centered, x_list, y_list, Rxencry_list, Ryencry_list = encode(origfpath, encodefpath) # type: ignore     
            # print(xencry_list, yencry_list, means, X_centered, Y_centered, x_list, y_list, Rxencry_list, Ryencry_list)
            # xdecry_list, ydecry_list = decode(encodefpath, means, X_centered, Y_centered, decodefpath)

            # perturbe_df = pd.read_csv(encodefpath)
            # orig_df = pd.read_csv(origfpath)
            estimate_df = pd.read_csv(decodefpath)

            # PerturbedList = perturbe_df["EncryPatchName"].tolist()
            # origList = orig_df["PatchName"].tolist()
            estList = estimate_df["DecryPatchName"].tolist()
            print(estList)

            subcoordinates = [find_decrpytcoordinates(label) for label in estList]
            print(subcoordinates)
            coordinates.extend(subcoordinates)
        de_coords.append(coordinates)

# patchName = 'wsi_8064_16800.png'

# print(find_decrpytcoordinates(patchName))
# print(find_encrpytcoordinates(patchName))
print(len(de_coords), de_coords)

In [ ]:
# 2. create sub data sets with splitting algorithm
from graph_based_splitting import graph_coloring_based_splitting, graph_coloring_based_splitting_visual, graph_coloring_based_splitting_visual1

# If ``strategy`` is a string, it must be one of the following,
# each of which represents one of the built-in strategy functions.

# * ``'largest_first'``
# * ``'random_sequential'``
# * ``'smallest_last'``
# * ``'independent_set'``
# * ``'connected_sequential_bfs'``
# * ``'connected_sequential_dfs'``
# * ``'connected_sequential'`` (alias for the previous strategy)
# * ``'saturation_largest_first'``
# * ``'DSATUR'`` (alias for the previous strategy)
# graph_distribution = []
# graph_split_file = os.mkdir(os.path.join(new_file_path, 'graph_split/'))

for coordinates, new_file_path in zip(de_coords, new_file_paths): 
    graph_split_file = f'{new_file_path}/graph_split1/'
    if not os.path.exists(graph_split_file):
        os.mkdir(graph_split_file)
    
    strategies = ['saturation_largest_first', 'largest_first', 'random_sequential', 'smallest_last', 'independent_set', 
            'connected_sequential']
    
    # strategies = ['independent_set', 'connected_sequential']
    for my_strategy in strategies: 
        # d, dataset_number = graph_coloring_based_splitting1(coordinates, patch_size, strategy=my_strategy)     # graph
        d, dataset_number = graph_coloring_based_splitting_visual1(graph_split_file, coordinates, patch_size, strategy=my_strategy)

        # # Specify the directory path
        # indexes = [item[0] for item in d.items()]
        # patch_folder = f'{new_file_path}/patches/'
        # patch_names = [os.path.join(patch_folder, patch) for patch in os.listdir(patch_folder)]

        # # Create a DataFrame from the list of file names
        # df = pd.DataFrame({"Index": indexes, "PatchName": patch_names})

        # # Specify the CSV file path where the DataFrame will be saved
        # csv_file_path = f'{new_file_path}/patch_names_{my_strategy}.csv'

        # # Save the DataFrame to a CSV file
        # df.to_csv(csv_file_path, index=False)
        # print(f'File names have been saved to {csv_file_path}')

        # OrgPatchName = df['PatchName']

        # # store the split image names into different .csv files
        # list = []
        # for j in range(dataset_number):
            
        #     indexes = []
        #     names = []
        #     for item in d.items(): 
        #         # print(item) 
        #         if item[1] == j:
        #             index = item[0]
        #             # print(i, index, OrgPatchName[index])
        #             indexes.append(index)
        #             names.append(OrgPatchName[index])
            
        #     dataframe = pd.DataFrame({"Index": indexes, "PatchName": names})
        #     destfp = os.path.join(graph_split_file, f'graph_patchesnames_'+str(j)+my_strategy+'.csv')
        #     print(destfp)
        #     # dataframe.to_csv('patchesnames_'+str(i)+'.csv', index=False, sep=',')
        #     dataframe.to_csv(destfp, index=False, sep=',')

    print("---------------Well Done----------------") 

# Evenly splitting

In [ ]:
from baseline_splitting import shuffle_evenly_division

# Evenly partitionly #
# * ``is_shuffle =  True``
# * ``is_shuffle =  False``
flag = True
dataset_numbers = [4, 5, 6, 7, 8]
for coords, new_file_path in zip(coordinates, new_file_paths): 
    for dataset_number in dataset_numbers:
        baseline_split_file = f'{new_file_path}/even_split/{dataset_number}/'
        if not os.path.exists(baseline_split_file):
            os.mkdir(baseline_split_file)
    
        # os.mkdir(graph_split_file)
        # res_path ='/root/code2024/dataset/2WSI/data/cnn_ensemble_updated/TCGA-A7-A6VX-01Z-00-DX2/even_split/'
        # patch_folder = '/root/code2024/dataset/2WSI/data/cnn_ensemble_updated/TCGA-A7-A6VX-01Z-00-DX2/patches/'
        patch_folder = f'{new_file_path}/patches/'
        patch_names = [os.path.join(patch_folder, patch) for patch in os.listdir(patch_folder)]

        # print(res_path)
        print(coordinates)
        # print(res.keys())
        
        # custom_keys = [key.replace(',', '_')+'.png' for key in res.keys()]

        
        is_shuffled = f'is_shuffled_{flag}'
        base_distribution = shuffle_evenly_division(patch_names, dataset_number, is_shuffle = flag)

        base_coords = shuffle_evenly_division(coordinates, dataset_number, is_shuffle = flag)
        print(base_coords)
        # for i in base_coords:
            # print(i)
        # indexes = [item[0] for item in d.items()]
        # patch_names = [os.path.join(patch_folder, patch) for patch in os.listdir(patch_folder)]

        list = []
        for j, item in enumerate(base_distribution): 
            indexes = []
            names = []
            print(item) 
            # for item in df['PatchName']:

            indexes = [i for i in range(len(item))]
            patch_names = [os.path.join(patch_folder, patch) for patch in item]

            dataframe = pd.DataFrame({"Indexes": indexes, "PatchName": patch_names})
            
            destfp = os.path.join(baseline_split_file, f'even_patchesnames_'+str(j)+f'_{is_shuffled}.csv')
            print(destfp)
            # dataframe.to_csv('patchesnames_'+str(i)+'.csv', index=False, sep=',')
            dataframe.to_csv(destfp, index=False, sep=',')

print("---------------Well Done----------------")  


# Even Splitting based Data Pertubation

In [ ]:
from encryption_r import encode, decode
    # r coordinates, new_file_path in zip(coords, new_file_paths): 
for new_file_path in new_file_paths: 
    strategy_nos = [4, 5, 6, 7, 8]
    for n in strategy_nos:
        even_split_file = f'{new_file_path}/even_split/{n}/'
        
        os.listdir(even_split_file)

        strategies = ['is_shuffled_True', 'is_shuffled_False']
        for my_strategy in strategies: 
            # my_strategy = 'smallest_last'
            # d, dataset_number = graph_coloring_based_splitting_visual1(graph_split_file, coordinates, patch_size, strategy=my_strategy)
            
            # 设置你的目标文件夹名称和文件模式
            # graph_split_file = f'{new_file_path}/graph_split1/'
            file_pattern = f"*[0-9]_{my_strategy}.csv"

            # 计算文件数量
            dataset_number = count_files_with_numeric_pattern(even_split_file, file_pattern)

            # print(f"在 '{target_folder_name}' 文件夹中找到了 {file_count} 个符合 '{file_pattern}' 模式的文件。")
            for j in range(dataset_number):
                # sub data set
                origfpath = f'{even_split_file}/even_patchesnames_{str(j)}_{my_strategy}.csv'
                encodefpath = f'{even_split_file}{my_strategy}_R_encode_coordinates_{j}.csv'
                decodefpath = f'{even_split_file}{my_strategy}_R_decode_coordinates_{j}.csv'

                print(new_file_path, my_strategy, j)
                xencry_list, yencry_list, means, X_centered, Y_centered, x_list, y_list, Rxencry_list, Ryencry_list = encode(origfpath, encodefpath) # type: ignore     
                # print(xencry_list, yencry_list, means, X_centered, Y_centered, x_list, y_list, Rxencry_list, Ryencry_list)
                xdecry_list, ydecry_list = decode(encodefpath, means, X_centered, Y_centered, decodefpath)

                perturbe_df = pd.read_csv(encodefpath)
                orig_df = pd.read_csv(origfpath)
                estimate_df = pd.read_csv(decodefpath)

                PerturbedList = perturbe_df["EncryPatchName"].tolist()
                origList = orig_df["PatchName"].tolist()
                estList = estimate_df["DecryPatchName"].tolist()



# Collect data

In [ ]:
import pandas as pd
from parse_coordinates import find_coordinates, find_encrpytcoordinates

import os
import re

simDa = ['graph_split1', 'even_split']
x = simDa[1]
coordfp = '/root/code2024/pipeline/YD/privacy_eval/coordinates/'

for i, new_file_path in enumerate(new_file_paths): 
    IGx = []
    IGy = []
    svg_path = ''

    encodefpath = []
    decodefpath = []
    origfpath = []

    if x == simDa[0]:
        # graph_split1
        graph_split_file = f'{new_file_path}/{x}/'
        fname = re.split(r'/', new_file_path)[-1]
        print(fname)
        # os.listdir(graph_split_file)
        strategies = ['largest_first', 'random_sequential', 'smallest_last', 'independent_set', 'connected_sequential', 'saturation_largest_first']
        for my_strategy in strategies: 
            file_pattern = f"*[0-9]{my_strategy}.csv"
            # 计算文件数量
            dataset_number = count_files_with_numeric_pattern(graph_split_file, file_pattern)
            for j in range(dataset_number):
                # sub data set
                origfpath = f'{graph_split_file}/graph_patchesnames_{str(j)}{my_strategy}.csv'
                encodefpath = f'{graph_split_file}{my_strategy}_R_encode_coordinates_{j}.csv'
                decodefpath = f'{graph_split_file}{my_strategy}_R_decode_coordinates_{j}.csv'
                svg_path = f'{coordfp}/{x}/{i}{fname}_coords_{my_strategy}_{j}.csv'

                perturbe_df = pd.read_csv(encodefpath)
                orig_df = pd.read_csv(origfpath)
                estimate_df = pd.read_csv(decodefpath)

                PerturbedList = perturbe_df["EncryPatchName"].tolist()
                origList = orig_df["PatchName"].tolist()
                estList = estimate_df["DecryPatchName"].tolist()
                print(len(PerturbedList), len(origList), len(estList))

                Dx = []
                Dy = []
                DPx = []
                DPy = []
                Decx = []
                Decy = []
                for d, dp, dec in zip(origList, PerturbedList, estList): 
                    # print(i,c,folder)
                    (xorg_coordinate, yorg_coordinate) = find_coordinates(d) # type: ignore
                    (x_coordinate, y_coordinate) = find_encrpytcoordinates(dp) # type: ignore
                    (xest_coordinate, yest_coordinate) = find_coordinates(dec) # type: ignore
                    Dx.append(int(xorg_coordinate))
                    Dy.append(int(yorg_coordinate))
                    DPx.append(float(x_coordinate))
                    DPy.append(float(y_coordinate))
                    Decx.append(int(xest_coordinate))
                    Decy.append(int(yest_coordinate))

                coords = {'orgcoord_x': Dx,
                        'orgcoord_y': Dy,
                        'percoord_x': DPx,
                        'percoord_y': DPy,  
                        'decoord_x': Decx,
                        'decoord_y': Decy
                        }
                Sdata = pd.DataFrame(coords)
                Sdata.to_csv(svg_path)

            print(i, my_strategy,"----------- Well Done----------")
    if x == simDa[1]:
        # Even partition
        dataset_numbers = [4, 5, 6, 7, 8]
        fname = re.split(r'/', new_file_path)[-1]
        print(fname)
    
        for dataset_number in dataset_numbers: 
            even_split_file = f'{new_file_path}/{x}/{dataset_number}/'
            os.listdir(even_split_file)
            strategies = ['is_shuffled_True', 'is_shuffled_False']
            for my_strategy in strategies: 
                file_pattern = f"*[0-9]_{my_strategy}.csv"

                # 计算文件数量
                dataset_number = count_files_with_numeric_pattern(even_split_file, file_pattern)

                for j in range(dataset_number):
                    # sub data set
                    origfpath = f'{even_split_file}/even_patchesnames_{str(j)}_{my_strategy}.csv'
                    encodefpath = f'{even_split_file}{my_strategy}_R_encode_coordinates_{j}.csv'
                    decodefpath = f'{even_split_file}{my_strategy}_R_decode_coordinates_{j}.csv'
                    svg_path = f'{coordfp}/{x}/{dataset_number}/{i}{fname}_coords_{my_strategy}_{j}.csv'
        
                    perturbe_df = pd.read_csv(encodefpath)
                    orig_df = pd.read_csv(origfpath)
                    estimate_df = pd.read_csv(decodefpath)

                    PerturbedList = perturbe_df["EncryPatchName"].tolist()
                    origList = orig_df["PatchName"].tolist()
                    estList = estimate_df["DecryPatchName"].tolist()
                    print(len(PerturbedList), len(origList), len(estList))

                    Dx = []
                    Dy = []
                    DPx = []
                    DPy = []
                    Decx = []
                    Decy = []
                    for d, dp, dec in zip(origList, PerturbedList, estList): 
                        # print(i,c,folder)
                        (xorg_coordinate, yorg_coordinate) = find_coordinates(d) # type: ignore
                        (x_coordinate, y_coordinate) = find_encrpytcoordinates(dp) # type: ignore
                        (xest_coordinate, yest_coordinate) = find_coordinates(dec) # type: ignore
                        Dx.append(int(xorg_coordinate))
                        Dy.append(int(yorg_coordinate))
                        DPx.append(float(x_coordinate))
                        DPy.append(float(y_coordinate))
                        Decx.append(int(xest_coordinate))
                        Decy.append(int(yest_coordinate))

                    coords = {'orgcoord_x': Dx,
                            'orgcoord_y': Dy,
                            'percoord_x': DPx,
                            'percoord_y': DPy,  
                            'decoord_x': Decx,
                            'decoord_y': Decy
                            }
                    Sdata = pd.DataFrame(coords)
                    Sdata.to_csv(svg_path)

                print(i, my_strategy,"----------- Well Done----------")

# Privacy Evaluation

In [ ]:
import numpy as np
import pandas as pd
import os
import statistics
from entropies import entropy1

def adjacent_values(vals, q1, q3):
    upper_adjacent_value = q3 + (q3 - q1) * 1.5
    upper_adjacent_value = np.clip(upper_adjacent_value, q3, vals[-1])

    lower_adjacent_value = q1 - (q3 - q1) * 1.5
    lower_adjacent_value = np.clip(lower_adjacent_value, vals[0], q1)
    return lower_adjacent_value, upper_adjacent_value


def set_axis_style(ax, labels):
    ax.set_xticks(np.arange(1, len(labels) + 1), labels=labels)
    ax.set_xlim(0.25, len(labels) + 0.75)
    # ax.set_xlabel('Sample name')

# 补齐数列
def bq(X,Y): 
    # Assuming len(y) < len(X)
    if len(X) < len(Y):
        reX = X + [0] * (abs(len(X) - len(Y)))
        X = reX
    elif len(X) > len(Y):
        reY = Y + [0] * (abs(len(X) - len(Y)))
        Y = reY    
    return X, Y


## NMI with original data

In [ ]:
simDa = ['graph_split1', 'even_split']
x = simDa[0]

fid = str(len(new_file_paths)-1)

privacy4strategies = {}    
if x == simDa[0]:
    coordfp = "/root/code2024/pipeline/YD/privacy_eval/coordinates/graph_split1/"
    strategies = ['largest_first', 'random_sequential', 'smallest_last', 'independent_set', 'connected_sequential', 'saturation_largest_first']

    print(fname)
    
    for my_strategy in strategies: 
        totfpath = os.path.join(coordfp, f'{fid}{fname}_coords_{my_strategy}.csv')
        _split_file = f'{coordfp}'
        # _split_file = f'{new_file_path}/{x}/'
        # 设置你的目标文件夹名称和文件模式
        file_pattern = f"*{fname}_coords_{my_strategy}_[0-9].csv"
        dataset_number = count_files_with_numeric_pattern(_split_file, file_pattern)
        subfno = [j for j in range(dataset_number)]
        
        all_dataX_per_s = []
        all_dataY_per_s = []

        IGx = []
        IGy = []

        NIx = []
        NIy = []
        
        for new_file_path in new_file_paths: 
            Dx = []
            Dy = []
            Dpx= []
            Dpy = []
            Dex= []
            Dey = []
            
            NMIX = []
            NMIY = []
            fname = re.split(r'/', new_file_path)[-1]
            for j in range(len(subfno)):
                ifilepath = os.path.join(coordfp, f'{fid}{fname}_coords_{my_strategy}_{j}.csv')
                
                # print(coordfp, ifilepath)
                idata = pd.read_csv(ifilepath)
            
                # print(idata)
                subfno.remove(j)
                rst = subfno
                print(fid, j, rst)
                cfilepath = [os.path.join(coordfp, f'{fid}{fname}_coords_{my_strategy}_{k}.csv') for k in rst]
                
                
                rstdata = [pd.read_csv(cfilepath[x]) for x in range(len(rst))]
                cdata = pd.concat(rstdata)

                X = idata['orgcoord_x'].tolist()
                Y = cdata['orgcoord_x'].tolist()
                ecodX = idata['percoord_x'].tolist()
                decodX = idata['decoord_x'].tolist()

                Z = idata['orgcoord_y'].tolist()
                W = cdata['orgcoord_y'].tolist()
                ecodZ = idata['percoord_y'].tolist()
                decodZ = idata['decoord_y'].tolist()

                # print("X", len(X), len(Y), len(Z), len(W))
                Dx.extend(X)
                Dy.extend(Z)
                
                Dpx.extend(ecodX)
                Dpy.extend(ecodZ)
                Dex.extend(decodX)
                Dey.extend(decodZ)
                
                
                # Calculate mutual information
                from scipy.stats.contingency import crosstab
                from sklearn.metrics import mutual_info_score, normalized_mutual_info_score, adjusted_mutual_info_score

                # output utility
                valuex,counts1x = np.unique(X, return_inverse=True)
                valuex,counts1dex = np.unique(decodX, return_inverse=True)
                
                valuex,counts1y = np.unique(Z, return_inverse=True)
                valuex,counts1dey = np.unique(decodZ, return_inverse=True)
            
                nmix_dex = normalized_mutual_info_score(counts1x, counts1dex)
                nmiy_dey = normalized_mutual_info_score(counts1y, counts1dey)
                # print(my_strategy, "individual output utility: ", nmix_dex, nmiy_dey)

                
                # individual privacy lower bound
                X, Y = bq(X,Y)
                Z, W = bq(Z,W)
            
                valuex,counts1x = np.unique(X, return_inverse=True)
                value2x,counts2x = np.unique(Y, return_inverse=True)

                valuex,counts1y = np.unique(Z, return_inverse=True)
                value2x,counts2y = np.unique(W, return_inverse=True)

                nmix = normalized_mutual_info_score(counts1x, counts2x)
                nmiy = normalized_mutual_info_score(counts1y, counts2y)
            
                rst.append(j)

                NMIX.append(nmix)
                NMIY.append(nmiy)

                # print(rst, subfno)
                # print(my_strategy, "individual privacy at lower bound: ", nmix, nmiy)
                # print('--------------')
            
            all_dataX_per_s.append(NMIX)
            all_dataY_per_s.append(NMIY)

            # print(f"NMI_{my_strategy}", NMIX, NMIY)
            # print(f"NMI_stat_{my_strategy}", statistics.mean(NMIX), statistics.stdev(NMIX), statistics.mean(NMIY), statistics.stdev(NMIY),'\n')

            # Calculate AIG
            x = entropy1(Dpx)-entropy1(Dx)
            y = entropy1(Dpy)-entropy1(Dy)
            IGx.append(x)
            IGy.append(y)
        
            # print(f"IG_{my_strategy}", IGx, IGy)
            # Calculate NI:lower bound on individual privacy
            # print(my_strategy, len(Dpx), len(Dx), len(Dpy), len(Dy))
            diffx_NI = [np.abs(Dpx[x] - Dx[x]) for x in range(len(Dx))] 
            diffy_NI = [np.abs(Dpy[x] - Dy[x]) for x in range(len(Dy))]     

            NIx.extend(diffx_NI)
            NIy.extend(diffy_NI)
        
        evalx_NI_std = np.std(NIx)
        evaly_NI_std = np.std(NIy) 
        evalx_NI_mean = np.mean(NIx)
        evaly_NI_mean = np.mean(NIy) 
        
        print(my_strategy, "NI(all data)", f'{evalx_NI_mean}+/-{round(evalx_NI_std, 8)}, {evaly_NI_mean}+/-{round(evaly_NI_std, 8)}')

        AIGx = round(sum(IGx)/len(IGx),8)
        AIGy = round(sum(IGy)/len(IGy),8)
        print(my_strategy, "AIG(all data)", round(AIGx, 4), round(AIGy, 4))

        eval_NMIX_mean = np.mean(all_dataX_per_s)
        eval_NMIY_mean = np.mean(all_dataY_per_s)
        eval_NMIX_std = np.std(all_dataX_per_s)
        eval_NMIY_std = np.std(all_dataY_per_s)
        
        privacy4strategies[my_strategy] = (round(eval_NMIX_mean, 3), round(eval_NMIY_mean,3))
        print(my_strategy, "NMI--individual privacy at lower bound(all data)", f'{round(eval_NMIX_mean, 3)}+/-{round(eval_NMIX_std, 3)}, {round(eval_NMIY_mean,3)}+/-{round(eval_NMIY_std, 3)}')
    print(privacy4strategies)

elif x == simDa[1]:
    dataset_numbers = [4,5,6,7,8]
    for dataset_number in dataset_numbers: 
        print(fname)
        coordfp = f"/root/code2024/pipeline/YD/privacy_eval/coordinates/even_split/{dataset_number}/"
        # print(coordfp)
        # strategies = ['is_shuffled_True', 'is_shuffled_False']
        # strategies = ['is_shuffled_False']
        strategies = ['is_shuffled_True']
        for my_strategy in strategies: 
            totfpath = os.path.join(coordfp, f'{fid}{fname}_coords_{my_strategy}.csv')
            _split_file = f'{coordfp}'
            # 设置你的目标文件夹名称和文件模式
            file_pattern = f"*{fname}_coords_{my_strategy}_[0-9].csv"
            dataset_number = count_files_with_numeric_pattern(_split_file, file_pattern)
            subfno = [j for j in range(dataset_number)]
            
            all_dataX_per_s = []
            all_dataY_per_s = []

            IGx = []
            IGy = []

            NIx = []
            NIy = []
            
            for new_file_path in new_file_paths: 
                Dx = []
                Dy = []
                Dpx= []
                Dpy = []
                Dex= []
                Dey = []
                
                NMIX = []
                NMIY = []
                fname = re.split(r'/', new_file_path)[-1]
                for j in range(len(subfno)):
                    ifilepath = os.path.join(coordfp, f'{fid}{fname}_coords_{my_strategy}_{j}.csv')
                    
                    # print(coordfp, ifilepath)
                    idata = pd.read_csv(ifilepath)
                
                    # print(idata)
                    subfno.remove(j)
                    rst = subfno
                    print(fid, j, rst)
                    cfilepath = [os.path.join(coordfp, f'{fid}{fname}_coords_{my_strategy}_{k}.csv') for k in rst]
                    
                    
                    rstdata = [pd.read_csv(cfilepath[x]) for x in range(len(rst))]
                    cdata = pd.concat(rstdata)

                    X = idata['orgcoord_x'].tolist()
                    Y = cdata['orgcoord_x'].tolist()
                    ecodX = idata['percoord_x'].tolist()
                    decodX = idata['decoord_x'].tolist()

                    Z = idata['orgcoord_y'].tolist()
                    W = cdata['orgcoord_y'].tolist()
                    ecodZ = idata['percoord_y'].tolist()
                    decodZ = idata['decoord_y'].tolist()

                    # print("X", len(X), len(Y), len(Z), len(W))
                    Dx.extend(X)
                    Dy.extend(Z)
                    
                    Dpx.extend(ecodX)
                    Dpy.extend(ecodZ)
                    Dex.extend(decodX)
                    Dey.extend(decodZ)
                    
                    
                    # Calculate mutual information
                    from scipy.stats.contingency import crosstab
                    from sklearn.metrics import mutual_info_score, normalized_mutual_info_score, adjusted_mutual_info_score

                    # output utility
                    valuex,counts1x = np.unique(X, return_inverse=True)
                    valuex,counts1dex = np.unique(decodX, return_inverse=True)
                    
                    valuex,counts1y = np.unique(Z, return_inverse=True)
                    valuex,counts1dey = np.unique(decodZ, return_inverse=True)
                
                    nmix_dex = normalized_mutual_info_score(counts1x, counts1dex)
                    nmiy_dey = normalized_mutual_info_score(counts1y, counts1dey)
                    # print(my_strategy, "individual output utility: ", nmix_dex, nmiy_dey)

                    
                    # individual privacy lower bound
                    X, Y = bq(X,Y)
                    Z, W = bq(Z,W)
                
                    valuex,counts1x = np.unique(X, return_inverse=True)
                    value2x,counts2x = np.unique(Y, return_inverse=True)

                    valuex,counts1y = np.unique(Z, return_inverse=True)
                    value2x,counts2y = np.unique(W, return_inverse=True)

                    nmix = normalized_mutual_info_score(counts1x, counts2x)
                    nmiy = normalized_mutual_info_score(counts1y, counts2y)
                
                    rst.append(j)

                    NMIX.append(nmix)
                    NMIY.append(nmiy)

                    # print(rst, subfno)
                    # print(my_strategy, "individual privacy at lower bound: ", nmix, nmiy)
                    # print('--------------')
                
                all_dataX_per_s.append(NMIX)
                all_dataY_per_s.append(NMIY)

                # print(f"NMI_{my_strategy}", NMIX, NMIY)
                # print(f"NMI_statis_{my_strategy}","individual privacy at lower bound: ", statistics.mean(NMIX), statistics.stdev(NMIX), statistics.mean(NMIY), statistics.stdev(NMIY),'\n')

                # Calculate AIG
                x = entropy1(Dpx)-entropy1(Dx)
                y = entropy1(Dpy)-entropy1(Dy)
                IGx.append(x)
                IGy.append(y)
            
                # print(f"IG_{my_strategy}", IGx, IGy)

                # Calculate NI:lower bound on individual privacy
                # print(my_strategy, len(Dpx), len(Dx), len(Dpy), len(Dy))
                diffx_NI = [np.abs(Dpx[x] - Dx[x]) for x in range(len(Dx))] 
                diffy_NI = [np.abs(Dpy[x] - Dy[x]) for x in range(len(Dy))]     

                NIx.extend(diffx_NI)
                NIy.extend(diffy_NI)
            
            evalx_NI_std = np.std(NIx)
            evaly_NI_std = np.std(NIy) 
            evalx_NI_mean = np.mean(NIx)
            evaly_NI_mean = np.mean(NIy) 
            
            # print(my_strategy, "NI(all data)", f'{evalx_NI_mean}+/-{round(evalx_NI_std, 8)}, {evaly_NI_mean}+/-{round(evaly_NI_std, 8)}')

            AIGx = round(sum(IGx)/len(IGx),8)
            AIGy = round(sum(IGy)/len(IGy),8)
            # print(my_strategy, "AIG(all data)", round(AIGx, 4), round(AIGy, 4))

            eval_NMIX_mean = np.mean(all_dataX_per_s)
            eval_NMIY_mean = np.mean(all_dataY_per_s)
            eval_NMIX_std = np.std(all_dataX_per_s)
            eval_NMIY_std = np.std(all_dataY_per_s)

            print(my_strategy, "NMI--individual privacy at lower bound(all data)", f'{round(eval_NMIX_mean, 3)}+/-{round(eval_NMIX_std, 3)}, {round(eval_NMIY_mean,3)}+/-{round(eval_NMIY_std, 3)}')
            privacy4strategies[(my_strategy, dataset_number)] = (round(eval_NMIX_mean, 3), round(eval_NMIY_mean,3))
            # print(my_strategy, "NMI--individual privacy at lower bound(all data)", f'{round(eval_NMIX_mean, 4)}+-{round(eval_NMIX_std, 4)}, {round(eval_NMIY_mean,4)}+-{round(eval_NMIY_std, 4)}')
            


## NMI with encoded data

In [ ]:
simDa = ['graph_split1', 'even_split']
x = simDa[0]

fid = str(len(new_file_paths)-1)

privacy4strategies = {}    
if x == simDa[0]:
    coordfp = "/root/code2024/pipeline/YD/privacy_eval/coordinates/graph_split1/"
    strategies = ['largest_first', 'random_sequential', 'smallest_last', 'independent_set', 'connected_sequential', 'saturation_largest_first']

    print(fname)
    
    for my_strategy in strategies: 
        totfpath = os.path.join(coordfp, f'{fid}{fname}_coords_{my_strategy}.csv')
        _split_file = f'{coordfp}'
        # _split_file = f'{new_file_path}/{x}/'
        # 设置你的目标文件夹名称和文件模式
        file_pattern = f"*{fname}_coords_{my_strategy}_[0-9].csv"
        dataset_number = count_files_with_numeric_pattern(_split_file, file_pattern)
        subfno = [j for j in range(dataset_number)]
        
        all_dataX_per_s = []
        all_dataY_per_s = []

        IGx = []
        IGy = []

        NIx = []
        NIy = []
        
        for new_file_path in new_file_paths: 
            Dx = []
            Dy = []
            Dpx= []
            Dpy = []
            Dex= []
            Dey = []
            
            NMIX = []
            NMIY = []
            fname = re.split(r'/', new_file_path)[-1]
            for j in range(len(subfno)):
                ifilepath = os.path.join(coordfp, f'{fid}{fname}_coords_{my_strategy}_{j}.csv')
                
                # print(coordfp, ifilepath)
                idata = pd.read_csv(ifilepath)
            
                # print(idata)
                subfno.remove(j)
                rst = subfno
                print(fid, j, rst)
                cfilepath = [os.path.join(coordfp, f'{fid}{fname}_coords_{my_strategy}_{k}.csv') for k in rst]
                
                
                rstdata = [pd.read_csv(cfilepath[x]) for x in range(len(rst))]
                cdata = pd.concat(rstdata)

                X = idata['orgcoord_x'].tolist()
                Y = cdata['orgcoord_x'].tolist()
                ecodX = idata['percoord_x'].tolist()
                ecodY = cdata['percoord_x'].tolist()

                decodX = idata['decoord_x'].tolist()
                decodY = cdata['decoord_x'].tolist()

                Z = idata['orgcoord_y'].tolist()
                W = cdata['orgcoord_y'].tolist()
                ecodZ = idata['percoord_y'].tolist()
                ecodW = cdata['percoord_y'].tolist()

                decodZ = idata['decoord_y'].tolist()
                decodW = cdata['decoord_y'].tolist()

                # print("X", len(X), len(Y), len(Z), len(W))
                Dx.extend(X)
                Dy.extend(Z)
                
                Dpx.extend(ecodX)
                Dpy.extend(ecodZ)
                Dex.extend(decodX)
                Dey.extend(decodZ)
                
                
                # Calculate mutual information
                from scipy.stats.contingency import crosstab
                from sklearn.metrics import mutual_info_score, normalized_mutual_info_score, adjusted_mutual_info_score

                # output utility
                valuex,counts1x = np.unique(X, return_inverse=True)
                valuex,counts1dex = np.unique(decodX, return_inverse=True)
                
                valuex,counts1y = np.unique(Z, return_inverse=True)
                valuex,counts1dey = np.unique(decodZ, return_inverse=True)
            
                nmix_dex = normalized_mutual_info_score(counts1x, counts1dex)
                nmiy_dey = normalized_mutual_info_score(counts1y, counts1dey)
                print(my_strategy, "individual output utility: ", nmix_dex, nmiy_dey)

                
                # individual privacy lower bound
                # X, Y = bq(X,Y)
                # Z, W = bq(Z,W)
            
                # valuex,counts1x = np.unique(X, return_inverse=True)     # original x(i)
                # value2x,counts2x = np.unique(Y, return_inverse=True)    # original x(j)

                # valuex,counts1y = np.unique(Z, return_inverse=True)     # original y(i)
                # value2x,counts2y = np.unique(W, return_inverse=True)    # # original y(j)

                # nmix = normalized_mutual_info_score(counts1x, counts2x)
                # nmiy = normalized_mutual_info_score(counts1y, counts2y)

                ecodX, ecodY = bq(X,Y)
                ecodZ, ecodW = bq(Z,W)
            

                valueedx,counts1edx = np.unique(ecodX, return_inverse=True)     # encode x(i)
                valueed2x,counts2edx = np.unique(ecodY, return_inverse=True)    # encode x(j)

                valueedx,counts1edy = np.unique(ecodZ, return_inverse=True)     # encode y(i)
                valueed2x,counts2edy = np.unique(ecodW, return_inverse=True)    # # encode y(j)

                nmix = normalized_mutual_info_score(counts1edx, counts2edx)
                nmiy = normalized_mutual_info_score(counts1edy, counts2edy)
            
                rst.append(j)

                NMIX.append(nmix)
                NMIY.append(nmiy)

                # print(rst, subfno)
                # print(my_strategy, "individual privacy at lower bound: ", nmix, nmiy)
                # print('--------------')
            
            all_dataX_per_s.append(NMIX)
            all_dataY_per_s.append(NMIY)

            # print(f"NMI_{my_strategy}", NMIX, NMIY)
            # print(f"NMI_stat_{my_strategy}", statistics.mean(NMIX), statistics.stdev(NMIX), statistics.mean(NMIY), statistics.stdev(NMIY),'\n')

            # Calculate AIG
            x = entropy1(Dpx)-entropy1(Dx)
            y = entropy1(Dpy)-entropy1(Dy)
            IGx.append(x)
            IGy.append(y)
        
            # print(f"IG_{my_strategy}", IGx, IGy)
            # Calculate NI:lower bound on individual privacy
            # print(my_strategy, len(Dpx), len(Dx), len(Dpy), len(Dy))
            diffx_NI = [np.abs(Dpx[x] - Dx[x]) for x in range(len(Dx))] 
            diffy_NI = [np.abs(Dpy[x] - Dy[x]) for x in range(len(Dy))]     
            # diffx_NI = [np.abs(Dex[x] - Dx[x]) for x in range(len(Dx))] 
            # diffy_NI = [np.abs(Dey[x] - Dy[x]) for x in range(len(Dy))]     


            NIx.extend(diffx_NI)
            NIy.extend(diffy_NI)
        
        evalx_NI_std = np.std(NIx)
        evaly_NI_std = np.std(NIy) 
        evalx_NI_mean = np.mean(NIx)
        evaly_NI_mean = np.mean(NIy) 
        
        print(my_strategy, "NI(all data)", f'{evalx_NI_mean}+/-{round(evalx_NI_std, 8)}, {evaly_NI_mean}+/-{round(evaly_NI_std, 8)}')

        AIGx = round(sum(IGx)/len(IGx),8)
        AIGy = round(sum(IGy)/len(IGy),8)
        print(my_strategy, "AIG(all data)", round(AIGx, 4), round(AIGy, 4))

        eval_NMIX_mean = np.mean(all_dataX_per_s)
        eval_NMIY_mean = np.mean(all_dataY_per_s)
        eval_NMIX_std = np.std(all_dataX_per_s)
        eval_NMIY_std = np.std(all_dataY_per_s)
        
        privacy4strategies[my_strategy] = (round(eval_NMIX_mean, 3), round(eval_NMIY_mean,3))
        print(my_strategy, "NMI--individual privacy at lower bound(all data)", f'{round(eval_NMIX_mean, 3)}+/-{round(eval_NMIX_std, 3)}, {round(eval_NMIY_mean,3)}+/-{round(eval_NMIY_std, 3)}')
    print(privacy4strategies)

elif x == simDa[1]:
    dataset_numbers = [4,5,6,7,8]
    for dataset_number in dataset_numbers: 
        print(fname)
        coordfp = f"/root/code2024/pipeline/YD/privacy_eval/coordinates/even_split/{dataset_number}/"
        # print(coordfp)
        # strategies = ['is_shuffled_True', 'is_shuffled_False']
        # strategies = ['is_shuffled_False']
        strategies = ['is_shuffled_True']
        for my_strategy in strategies: 
            totfpath = os.path.join(coordfp, f'{fid}{fname}_coords_{my_strategy}.csv')
            _split_file = f'{coordfp}'
            # 设置你的目标文件夹名称和文件模式
            file_pattern = f"*{fname}_coords_{my_strategy}_[0-9].csv"
            dataset_number = count_files_with_numeric_pattern(_split_file, file_pattern)
            subfno = [j for j in range(dataset_number)]
            
            all_dataX_per_s = []
            all_dataY_per_s = []

            IGx = []
            IGy = []

            NIx = []
            NIy = []
            
            for new_file_path in new_file_paths: 
                Dx = []
                Dy = []
                Dpx= []
                Dpy = []
                Dex= []
                Dey = []
                
                NMIX = []
                NMIY = []
                fname = re.split(r'/', new_file_path)[-1]
                for j in range(len(subfno)):
                    ifilepath = os.path.join(coordfp, f'{fid}{fname}_coords_{my_strategy}_{j}.csv')
                    
                    # print(coordfp, ifilepath)
                    idata = pd.read_csv(ifilepath)
                
                    # print(idata)
                    subfno.remove(j)
                    rst = subfno
                    print(fid, j, rst)
                    cfilepath = [os.path.join(coordfp, f'{fid}{fname}_coords_{my_strategy}_{k}.csv') for k in rst]
                    
                    
                    rstdata = [pd.read_csv(cfilepath[x]) for x in range(len(rst))]
                    cdata = pd.concat(rstdata)

                    X = idata['orgcoord_x'].tolist()
                    Y = cdata['orgcoord_x'].tolist()
                    ecodX = idata['percoord_x'].tolist()
                    decodX = idata['decoord_x'].tolist()

                    Z = idata['orgcoord_y'].tolist()
                    W = cdata['orgcoord_y'].tolist()
                    ecodZ = idata['percoord_y'].tolist()
                    decodZ = idata['decoord_y'].tolist()

                    # print("X", len(X), len(Y), len(Z), len(W))
                    Dx.extend(X)
                    Dy.extend(Z)
                    
                    Dpx.extend(ecodX)
                    Dpy.extend(ecodZ)
                    Dex.extend(decodX)
                    Dey.extend(decodZ)
                    
                    
                    # Calculate mutual information
                    from scipy.stats.contingency import crosstab
                    from sklearn.metrics import mutual_info_score, normalized_mutual_info_score, adjusted_mutual_info_score

                    # output utility
                    valuex,counts1x = np.unique(X, return_inverse=True)
                    valuex,counts1dex = np.unique(decodX, return_inverse=True)
                    
                    valuex,counts1y = np.unique(Z, return_inverse=True)
                    valuex,counts1dey = np.unique(decodZ, return_inverse=True)
                
                    nmix_dex = normalized_mutual_info_score(counts1x, counts1dex)
                    nmiy_dey = normalized_mutual_info_score(counts1y, counts1dey)
                    # print(my_strategy, "individual output utility: ", nmix_dex, nmiy_dey)

                    
                    # individual privacy lower bound
                    X, Y = bq(X,Y)
                    Z, W = bq(Z,W)
                
                    valuex,counts1x = np.unique(X, return_inverse=True)
                    value2x,counts2x = np.unique(Y, return_inverse=True)

                    valuex,counts1y = np.unique(Z, return_inverse=True)
                    value2x,counts2y = np.unique(W, return_inverse=True)

                    nmix = normalized_mutual_info_score(counts1x, counts2x)
                    nmiy = normalized_mutual_info_score(counts1y, counts2y)
                
                    rst.append(j)

                    NMIX.append(nmix)
                    NMIY.append(nmiy)

                    # print(rst, subfno)
                    # print(my_strategy, "individual privacy at lower bound: ", nmix, nmiy)
                    # print('--------------')
                
                all_dataX_per_s.append(NMIX)
                all_dataY_per_s.append(NMIY)

                # print(f"NMI_{my_strategy}", NMIX, NMIY)
                # print(f"NMI_statis_{my_strategy}","individual privacy at lower bound: ", statistics.mean(NMIX), statistics.stdev(NMIX), statistics.mean(NMIY), statistics.stdev(NMIY),'\n')

                # Calculate AIG
                x = entropy1(Dpx)-entropy1(Dx)
                y = entropy1(Dpy)-entropy1(Dy)
                IGx.append(x)
                IGy.append(y)
            
                # print(f"IG_{my_strategy}", IGx, IGy)

                # Calculate NI:lower bound on individual privacy
                # print(my_strategy, len(Dpx), len(Dx), len(Dpy), len(Dy))
                # diffx_NI = [np.abs(Dpx[x] - Dx[x]) for x in range(len(Dx))] 
                # diffy_NI = [np.abs(Dpy[x] - Dy[x]) for x in range(len(Dy))]     

                diffx_NI = [np.abs(Dx[x] - Dx[x]) for x in range(len(Dx))] 
                diffy_NI = [np.abs(Dy[x] - Dy[x]) for x in range(len(Dy))]     

                NIx.extend(diffx_NI)
                NIy.extend(diffy_NI)
            
            evalx_NI_std = np.std(NIx)
            evaly_NI_std = np.std(NIy) 
            evalx_NI_mean = np.mean(NIx)
            evaly_NI_mean = np.mean(NIy) 
            
            # print(my_strategy, "NI(all data)", f'{evalx_NI_mean}+/-{round(evalx_NI_std, 8)}, {evaly_NI_mean}+/-{round(evaly_NI_std, 8)}')

            AIGx = round(sum(IGx)/len(IGx),8)
            AIGy = round(sum(IGy)/len(IGy),8)
            # print(my_strategy, "AIG(all data)", round(AIGx, 4), round(AIGy, 4))

            eval_NMIX_mean = np.mean(all_dataX_per_s)
            eval_NMIY_mean = np.mean(all_dataY_per_s)
            eval_NMIX_std = np.std(all_dataX_per_s)
            eval_NMIY_std = np.std(all_dataY_per_s)

            print(my_strategy, "NMI--individual privacy at lower bound(all data)", f'{round(eval_NMIX_mean, 3)}+/-{round(eval_NMIX_std, 3)}, {round(eval_NMIY_mean,3)}+/-{round(eval_NMIY_std, 3)}')
            privacy4strategies[(my_strategy, dataset_number)] = (round(eval_NMIX_mean, 3), round(eval_NMIY_mean,3))
            # print(my_strategy, "NMI--individual privacy at lower bound(all data)", f'{round(eval_NMIX_mean, 4)}+-{round(eval_NMIX_std, 4)}, {round(eval_NMIY_mean,4)}+-{round(eval_NMIY_std, 4)}')
            


In [ ]:
N = []
startegies = []
x_privacy = []
y_privacy = []
label = 'is_shuffled_True'
print(privacy4strategies)
for i in privacy4strategies.keys():
    if i[0] == label:
        print(i, privacy4strategies[i])
        N.append(i[1])
        startegies.append(i)
        x_privacy.append(privacy4strategies[i][0])
        y_privacy.append(privacy4strategies[i][1])


# 相同的份数，不同的策略

# 双坐标轴图 -- 统计分析individual privacy